In [1]:
import numpy as np
import dash
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html
import plotly
import random
import plotly.graph_objs as go
from collections import deque

from multiprocessing import Process, Queue
import threading
from data_collect import DataCollect
from serial_data import SerialData

from scipy import signal
from utils.QRS_util import *

import matplotlib.pyplot as plt

In [2]:
timestep = deque(maxlen = 10000)
timestep.append(0)

raw_ECG = deque(maxlen = 10000)
raw_ECG.append(0)

proc_ECG = deque(maxlen = 10000)
proc_ECG.append(0)

QRS_pnts = deque(maxlen = 10000)
QRS_pnts.append('yellow')

QRS_sizes = deque(maxlen = 10000)
QRS_sizes.append(1)

bpm_ECG = [0]

In [3]:
app = dash.Dash(__name__)

app.layout = html.Div(children = 
    [
        html.H1(id = 'bpm-val', children = ["init"]),
        dcc.Interval(
            id = 'bpm-val-update',
            disabled=False,
            interval = 3000,
            n_intervals = 0
        ),
        dcc.Graph(id = 'raw-ecg', animate = True),
        dcc.Interval(
            id = 'raw-ecg-update',
            interval = 3000,
            n_intervals = 0
        ),
        dcc.Graph(id = 'proc-ecg', animate = True),
        dcc.Interval(
            id = 'proc-ecg-update',
            interval = 3000,
            n_intervals = 0
        ),
    ]
)

@app.callback(
    Output('bpm-val', 'children'),
    [ Input("bpm-val-update", "n_intervals") ]
)
def update_bpm(n):
    
    return f"BPM: {bpm_ECG[0]:.1f}"

@app.callback(
    Output('raw-ecg', 'figure'),
    [ Input('raw-ecg-update', 'n_intervals') ]
)
def update_raw_ecg(n):

    data = plotly.graph_objs.Scatter(
            x=list(timestep),
            y=list(raw_ECG),
            name='Scatter',
            mode='lines',
            #marker_color=[1 if x%2==0 else 0 for x in np.arange(len(X))]
    )

    return {'data': [data],
            'layout' : go.Layout(xaxis=dict(range=[min(timestep),max(timestep)]),yaxis = dict(range = [min(raw_ECG),max(raw_ECG)]),)}

@app.callback(
    Output('proc-ecg', 'figure'),
    [ Input('proc-ecg-update', 'n_intervals') ]
)
def update_proc_ecg(n):

    data = plotly.graph_objs.Scatter(
            x=list(timestep),
            y=list(proc_ECG),
            name='Scatter',
            mode='lines+markers',
            marker_color=list(QRS_pnts),
            line_color='rgb(0,0,0)',
            marker_size = list(QRS_sizes),
    )

    return {'data': [data],
            'layout' : go.Layout(xaxis=dict(range=[min(timestep),max(timestep)]),yaxis = dict(range = [min(proc_ECG),max(proc_ECG)]),)}

In [4]:
fs = 1000
nseg = 256
amp = 2 * np.sqrt(2)

def remove_outliers(data):
    q_25 = np.quantile(data, 0.25)
    q_75 = np.quantile(data, 0.75)
    q_50 = np.quantile(data, 0.5)
    upper_limit = q_50 + 10 * (q_75 - q_25)
    lower_limit = q_50 - 10 * (q_75 - q_25)
    data = np.where(data < upper_limit, data, q_50)
    data = np.where(data > lower_limit, data, q_50)
    return data

def normalize(data):
    return (data - data.mean()) / (data.std() + 0.000000001)

def process_ecg(data, plot = False):
    data = remove_outliers(data)
    
    data = normalize(data)
    signal_min = data.min()
    signal_max = data.max()
    
    f, t, Zxx = signal.stft(data, fs, nperseg = nseg)
    Zxx_new = Zxx.copy()
    Zxx_new[25:, :] = 0
    
    _, new_data = signal.istft(Zxx_new, fs)
    new_data = np.where(new_data > signal_min, new_data, signal_min)
    new_data = np.where(new_data < signal_max, new_data, signal_max)
    new_data = normalize(new_data)
    new_data = np.where(new_data < 0, - new_data ** 2, new_data ** 3)
    new_data = normalize(new_data) * 25
    
    if(plot == True):
        fig, ax = plt.subplots(2, 1, figsize = (20, 15), sharex = True)
        ax[0].plot(data)
        ax[0].set_title("Original Signal", fontsize = 20)
        ax[1].plot(new_data)
        ax[1].set_xticks(np.arange(0, data.shape[0] // fs * fs + 1, fs))
        ax[1].set_xticklabels([str(x) for x in np.arange(0, data.shape[0] // fs + 1)])
        ax[1].set_xlabel("Time (s)", fontsize = 15)
        ax[1].set_title("Processed Signal", fontsize = 20)
        plt.show()
    
    return new_data

In [5]:
def task1():
    app.run_server()

def task2():
    data_collect = DataCollect()
    serial_data = SerialData()

    # S1 to S2 communication
    queueS1 = Queue()  # s1.stage1() writes to queueS1

    # S2 to S1 communication
    queueS2 = Queue()  # s2.stage2() writes to queueS2

    # start s2 as another process
    s2 = Process(target = serial_data.serial_data, args=(queueS1, queueS2))
    s2.daemon = True
    s2.start()     # Launch the stage2 process

    for msg in data_collect.data_collect(queueS1, queueS2):
        prev_timestep = timestep[-1]
        timestep.extend([prev_timestep + x for x in range(len(msg))])
        raw_ECG.extend(msg)
        #plt.plot(msg)
        #plt.show()
        
        proc_msg = process_ecg(msg, plot = False)
        proc_ECG.extend(proc_msg)
        #plt.plot(proc_msg)
        #plt.show()
        
        R_pnts, S_pnts, Q_pnts = EKG_QRS_detect(proc_msg, 1000, True, False, factor = 3)
        if(len(R_pnts) == 0):
            colors = ['rgb(0,0,0)' for x in range(len(proc_msg))]
        else:
            def check_QRS(pnt):
                if(pnt in Q_pnts):
                    return 'rgb(255,0,0)'
                elif(pnt in R_pnts):
                    return 'rgb(0,255,0)'
                elif(pnt in S_pnts):
                    return 'rgb(0,0,255)'
                else:
                    return 'rgb(0,0,0)'
            colors = [check_QRS(x) for x in range(len(proc_msg))]
        QRS_pnts.extend(colors)
        def color_size_map(x):
            if(x == 'rgb(0,255,0)'):
                return 30
            elif((x == 'rgb(255,0,0)') | (x == 'rgb(0,0,255)')):
                return 10
            else:
                return 1
        QRS_sizes.extend([color_size_map(x) for x in colors])
        #plt.scatter(x = np.arange(len(proc_msg)), y = proc_msg, c = colors)
        #plt.show()
        
        bpm_ECG[0] = (np.sum([x=='rgb(0,255,0)' for x in QRS_pnts]) / (len(QRS_pnts) / 1000.0) * 60.0)
    
    s2.join() # wait till s2 daemon finishes

In [6]:
if(__name__ == '__main__'):
    p1 = threading.Thread(target = task1)
    p1.start()
    p2 = threading.Thread(target = task2)
    p2.start()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Apr/2021 23:04:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2021 23:04:18] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2021 23:04:18] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2021 23:04:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2021 23:04:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2021 23:04:18] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2021 23:04:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2021 23:04:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2021 23:04:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2021 23:04:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2021 23:04:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2021 23:04:24] "POST /_dash-update-component